# Potential Outcomes 

The concept of potential outcomes are foundational to the reasoning system that we're going to use for the course. They allow us to think about, with considerably clarity, the comparisons that we want to make, and **exactly** why these comparisons are *causal*. 

But, despite the somewhat obtuse language that we might use when we're talking about potential outcomes (e.g. "The potential outcome to treatment, when a unit is assigned to treatment.") the math surrounding potential outcomes is actually pretty straightforward. 

# Create Data 
Let's make some toy data that we can use through this demo. 

Suppose that we have 1,000 individuals that we can observe in our study. Furthermore, suppose that for each of these people, there is some *latent*, or unmeasured, response to treatment that they *would* have to treatment if we assigned them to take the treatemnt. 

You might think of this as an unknowable population-level parameter that we're trying to estimate from empirical data. In this worksheet, we're going to make the data so that the causal parameter $\tau$ has a mean value of 2. But, you could imagine that some other treatment has a mean value of 100. Or, for some third treatment it could have a mean value of zero, in which case we would say there there is *no* treatment effect. 

For concreteness, suppose that the treatment is assigning people to drink [coffee](https://www.aldeacoffee.com). 


Create a data.table with the following columns: 

1. An `id` column that ranges from 1 to the number of rows in the data.table. 
2. A `tau` column that has random draws from the normal distribution. Set the mean of these draws to be two, and the standard deviation to also be two. 

In [1]:
library(data.table)

NROWS <- 1000 

d <- data.table(
  'id'  = 1:NROWS, 
  'tau' = rnorm(n = NROWS, mean = 2, sd = 2) 
)

Let's build more into our data, like, that some of the people are tall, while others are short; and some are young, while others are old. 

Make two new colums, using the `:=` assignment operator in the data.table. We'll make the first column for you. You produce the second column. 

1. `Height` which is random draws of either 'tall' or 'short'. 
2. `Age` which is random draws of either 'young' or 'old'. 


In [2]:
d[ , height := sample(c('tall','short'), size = .N, replace = TRUE)]
d[ , age    := sample(c('young', 'old'), size = .N, replace = TRUE)]

In just exactly the same way, we can also think of our units having levels of *potential outcomes to control*. That is, we can suppose that people have some level of the outcome in the case that they do not drink any coffee. For concreteness, suppose that the outcome is people's number of minutes of 241 coding they can accomplish, before they fall asleep. 

There might (or might not) be a relationship between our non-experimentally assigned outcomes. In the example we're working with here, suppose that there is no relationship between height and minutes of coding; but that there is a positive relationship between age and minutes of coding. 

This might be represented in our data in the following way: The mean number of minutes that someone can work is 10, plus, if they're old (ahem... seasoned, disciplined) they are able to work for an additional 5 minutes on avergae. But, also assume that there is some noise in this relationship. 

Create a column on the data called `y_0` that takes the following values. 

- If the preson is young, it takes the value of 10. 
- If the person is old, it takes the value of 15. 
- For both young and old people add in a little bit of *noise* a a draw from a random normal distribution with `mean=0` and `sd=1`. 

There are a couple of ways that you can do this. While you may be tempted to subset your data; this is probably less efficient that it could be. 

- Either use an `ifelse` statement, which is still not maximally performant; or, 
- Use an evaluation statement using the `I()` "as is" operator. 

In [3]:
d[ , y_0 := 10 + I(age == 'old') * 5 + rnorm(.N, mean = 0, sd = 1)]

Notice that there is no relationship between height and potential outcomes to control; and also notice that the "noise" in the relationship is represented in the draw from the normal distribution with mean 0. 

## Potential Outcomes to Treatment 
If we know people's potential outcomes to control, and we already know each persons *causal effect*, then I suppose we also know their potential outcomes to **treatment**, right? 

Create people's potential outcomes to treatment, `y_1` as the sum of their `y_0` and `tau` values. 

In [4]:
d[ , y_1 := y_0 + tau]

And so, we can represent this **science table** -- the set of all potentially realizable outcomes, and also measurable covariates. 

In [5]:
head(d)

id,tau,height,age,y_0,y_1
1,1.3616870,short,old,13.95007....,15.31176....
2,-0.3519366,tall,old,16.27610....,15.92417....
3,1.7194282,short,young,8.589853....,10.30928....
4,-1.1878242,short,young,11.25768....,10.06986....
5,1.5952862,short,old,15.13347....,16.72876....
6,0.5188273,tall,young,10.20566....,10.72449....


## Questions for Understanding 

First, answer these based on your understanding of how you created the data. Then, answer them using calls and subset against the data. This is *easy* practice for querying the data.tables and performing functions on them. 

1. On average, will people who are older tend to be taller, shorter, or about the same height, and people who are younger? 
2. On average, will people who are older tend to have higher or lower potential outcomes to control? 
3. On average, will people who are older tend to have higher or lower potential outcomes to treatment? 
4. **Most importantly**: If these are *potential outcomes* then, can we empirically observe any of these outcomes? 


In [6]:
d[ , .(mean_tall = mean(height == 'tall')), by = .(age)]

age,mean_tall
old,0.5000000
young,0.4820717


In [7]:
d[ , .(mean_control = mean(y_0)), by = .(age)]

age,mean_control
old,15.05496....
young,10.02477....


In [8]:
d[, .(mean_treat = mean(y_1)), by = .(height)]

height,mean_treat
short,14.47931....
tall,14.54258....


# Run Your Experiment 

To this point, we've been working in the *potential outcomes space*. One way that you might think about this is as though these are the measurement that our population of people are going to walk into the expeiment having -- but we haven't measured them yet, so we don't know what they are! That is, suppose we have the whole cohort of students who are enrolled in 241, and they're going to start the coffee drinking experiment in Week 3. In Week 2, they all are either young/old, short/tall, and have innate abilities to focus, *we as the experimenters just don't know them yet!*. 

When we run the experiment, we accomplish several things: 

1. We measure outcomes from our subjects;
2. We intervene in their lives to force a particular experience; 
3. As a result of our intervention, we *reveal* either potential outcomes to treatment or control for each subject, and we measure this. 

The first of these is easy: For every subject, no matter whether they are in treatment or control, we measure the trait that we care about. But, the second two require consierably more care, and are the focus of the class. 

In this experiment, the way that we are interviening in people's lives is to either give them *coffee* or *decaf coffee*. Then, and only then, we're also able to make some of their potential outcomes measurable. 


Make two final columns: 

1. Create variable `treat` and randomly assign people to be either in the treatment group or the control group. 
2. Create a variable `Y` that is the potential outcome that we *actually* observe in the experiment. If the person is assigned to the treatment group, fill `Y = y_1`; and, if they're assigned to the control group fill `Y = y_0`. 

In [9]:
d[ , treat := sample(c(0,1), size = .N, replace = TRUE)]
d[ , Y := ifelse(treat, y_1, y_0)]
head(d)

id,tau,height,age,y_0,y_1,treat,Y
1,1.3616870,short,old,13.95007....,15.31176....,0,13.95007
2,-0.3519366,tall,old,16.27610....,15.92417....,0,16.27611
3,1.7194282,short,young,8.589853....,10.30928....,1,10.30928
4,-1.1878242,short,young,11.25768....,10.06986....,0,11.25769
5,1.5952862,short,old,15.13347....,16.72876....,1,16.72876
6,0.5188273,tall,young,10.20566....,10.72449....,1,10.72449


## Why only some of? 

Why are we only able to measure some of people's potential outcomes? As David has identified in the async videos, we're only able to measure the potential outcomes that are consistent with the treatment that we actually give people. 

And, so, while the *science table* might contain information about each persons potential outcomes to treamtent and control, the table of data that we're ever going to be able to generate has a more restricted set. Call this observable data set `obs`, and it is a subset of all the data that might be out there.

In [10]:
obs <- d[ , .(id, height, age, treat, Y)]
head(obs)

id,height,age,treat,Y
1,short,old,0,13.95007
2,tall,old,0,16.27611
3,short,young,1,10.30928
4,short,young,0,11.25769
5,short,old,1,16.72876
6,tall,young,1,10.72449


Where did all the data go?? We no longer have access to people's causal effect ($\tau$), nor do we have access to their potential outcomes to control and treatment. Instead, we're left only with their *realized potential outcomes* that match the condition they were assigned to. 

## Question of Understanding (Conceptual)

1. What happens to these values that are no longer in our dataset? Because we can't measure them, does that mean that they don't exist? Does it mean that they *never* existed? 
2. If we can't see any person's $\tau$, then how should we generate an estimate of this population-level parameter? 

Just because we can't measure them, doesn't mean they don't exist! They just become counterfactual. And, even though we cannot see a person's individual treatment effect, if we can generate an unbiased statement of the populations potential outcomes to treatment and control, then we can generate an unbaised estimate for each persons' individual $\tau$. 

## Questions of Understanding (Numerical) 

Use the data.table `d` for these questions. 

1. Is the average height of people who are in treatment more or less than the average height for people who are in control? 
2. Among poeple who are in treatment, what is the mean potential outcome to control (`y_0`)? Is this any different than the people who are in control? 
3. Among people who are in treatment, what is the mean potential outcome to treatment (`y_1`)? Is this any different than the people who are control? 
4. Among people who are in treatment, what is the average difference between the potential outcomes to treatment and potential outcomes to control? 

In [11]:
d[ , .(mean_height = mean(height=='tall')), by = .(treat)]

treat,mean_height
0,0.4920319
1,0.4899598


In [12]:
d[ , .(mean_y0 = mean(y_0)), by = .(treat)]

treat,mean_y0
0,12.42817....
1,12.63225....


In [13]:
d[ , .(mean_y1 = mean(y_1)), by = .(treat)]

treat,mean_y1
0,14.37685....
1,14.64498....


In [14]:
d[treat==1, .(mean_y1 = mean(y_1), 
              mean_y0 = mean(y_0))]

mean_y1,mean_y0
14.64499,12.63226


# Estimating Causal Quantities from the Observable Data 

Of course, we don't get to see each of these potential outcomes
instead, we only get to see either one or the other. Which one do we
get to see? Well the potential outcome for the treatment condition that
we assign the person to! 

But, though random assignment, the values for the potential outcomes to control that we see *should* be an unbiased estiamte of the whole populations potential outcomes to control. 

In [15]:
mean_y0        <- d[ , mean(y_0)]
mean_control_Y <- obs[treat==0, mean(Y)]

sprintf("The mean of y0 is %.3f, and the mean of contorl Y is %.3f.", mean_y0, mean_control_Y)

[1] "The mean of y0 is 12.530, and the mean of contorl Y is 12.428."

Conduct the same comparison among the treatment group. Does the result surprise you at all? 

In [16]:
mean_y1      <- d[ , mean(y_0)]
mean_treat_Y <- obs[treat==1, mean(Y)]

sprintf("The mean of y1 is %.3f, and the mean of treatment Y is %.3f.", mean_y1, mean_treat_Y)

[1] "The mean of y1 is 12.530, and the mean of treatment Y is 14.645."

# The Big Punchline! 

Because the *observable* set of realised potential outcomes are unbiased estimates of the *unobservable* potential outcomes, we're able to generate an unbiased estimate of the causal effect, using only obsevable data! 

- Calculate the mean of the observable potential outcomes to treatment. 
- Calcualte the mean of the observable potential outcomes to control. 
- Then, calculate the difference between the two of these quantities. 
- How close is this $\hat{\tau}$ to the true $\tau$ that you created in the first code cell? 

In [17]:
treat_mean   <- d[treat==1, mean(Y)]
control_mean <- d[treat==0, mean(Y)]

treat_mean - control_mean

[1] 2.216811

Let that sink in a bit. 

Let that sink in a bit. Most of your job as a data scientist *not* in producing some results; most of your job as a data scientist is in producing reuslts that are the *right* results, and that requires thought. 